https://www.kaggle.com/code/siddigantm/task-resume-matching-with-job-descriptions/input?select=training_data.csv

In [26]:
import pandas as pd
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from torch.nn.functional import cosine_similarity


# HH Data

## Resume

In [2]:
df_resume = pd.read_csv('dst-3.0_16_1_hh_database.csv', delimiter=';')

In [3]:
df_resume['Ищет работу на должность:'] = df_resume['Ищет работу на должность:'].str.split(', ')
df_resume = df_resume.explode('Ищет работу на должность:')
df_resume['Ищет работу на должность:'] = df_resume['Ищет работу на должность:'].str.lower()
df_resume = df_resume.reset_index()

## Vacancies

In [4]:
df_it_vacancies = pd.read_csv('IT_vacancies_full.csv')

In [5]:
df_it_vacancies['Name'] = df_it_vacancies['Name'].str.lower()
job_resume = df_it_vacancies.merge(df_resume, left_on='Name', right_on='Ищет работу на должность:', how='left')
job_resume = job_resume.dropna(subset=['Ищет работу на должность:'])
job_resume = job_resume.drop_duplicates(subset=['Ids', 'index'])

In [6]:
counts = job_resume['Ids'].value_counts()
counts = counts[counts >= 10]
to_keep = counts.index
job_resume = job_resume[job_resume['Ids'].isin(to_keep)]

In [27]:
job_resume['Description'] = job_resume['Description'].astype(str)
job_resume['Опыт работы'] = job_resume['Опыт работы'].astype(str)

In [37]:
job_resume

,Ids,Employer,Name,Salary,From,To,Experience,Schedule,Keys,Description,...,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
21,49845512,Комплексные Коммуникации,менеджер проектов,True,73950.0,130500.0,От 3 до 6 лет,Полный день,"['Готовность к командировкам', 'Работа в услов...",Компания «Комплексные Коммуникации» - более 15...,...,менеджер проектов,"Томск , не готова к переезду , готова к редким...",полная занятость,"гибкий график, полный день, удаленная работа",Опыт работы 2 года 6 месяцев Менеджер проекто...,"ООО""Спар-Томск""",Менеджер по продвижению,Неоконченное высшее образование 2021 Национал...,22.04.2019 16:51,Не указано
22,49845512,Комплексные Коммуникации,менеджер проектов,True,73950.0,130500.0,От 3 до 6 лет,Полный день,"['Готовность к командировкам', 'Работа в услов...",Компания «Комплексные Коммуникации» - более 15...,...,менеджер проектов,"Москва , м. Фили , не готова к переезду , гот...",полная занятость,полный день,Опыт работы 14 лет 6 месяцев Февраль 2016 — п...,АО «ПФ «СКБ Контур»,Менеджер по работе с ключевыми клиентами,"Высшее образование 2008 ""Санкт-Петербургская а...",18.04.2019 20:33,Не указано
23,49845512,Комплексные Коммуникации,менеджер проектов,True,73950.0,130500.0,От 3 до 6 лет,Полный день,"['Готовность к командировкам', 'Работа в услов...",Компания «Комплексные Коммуникации» - более 15...,...,менеджер проектов,"Москва , м. Измайловская , не готов к переезд...",полная занятость,гибкий график,Опыт работы 6 лет 7 месяцев Менеджер проектов...,ЕВРО ПОИНТ,Заместитель руководителя операционного отдела,Неоконченное высшее образование 2019 Московск...,26.04.2019 12:24,Не указано
24,49845512,Комплексные Коммуникации,менеджер проектов,True,73950.0,130500.0,От 3 до 6 лет,Полный день,"['Готовность к командировкам', 'Работа в услов...",Компания «Комплексные Коммуникации» - более 15...,...,менеджер проектов,"Краснодар , не готов к переезду , готов к кома...","частичная занятость, проектная работа, полная ...","полный день, сменный график, удаленная работа",Опыт работы 11 лет 2 месяца Октябрь 2016 — Ап...,"ООО ""Полимертрейд""",Региональный менеджер по продажам,Высшее образование 2005 Кубанский государстве...,13.04.2019 09:56,Не указано
25,49845512,Комплексные Коммуникации,менеджер проектов,True,73950.0,130500.0,От 3 до 6 лет,Полный день,"['Готовность к командировкам', 'Работа в услов...",Компания «Комплексные Коммуникации» - более 15...,...,менеджер проектов,"Санкт-Петербург , м. Академическая , готов к ...","стажировка, частичная занятость, проектная раб...","гибкий график, полный день, сменный график",Опыт работы 5 лет 7 месяцев Март 2017 — по на...,"ПАО ""Ростелеком""",Менеджер проектов,Высшее образование 2013 Санкт-Петербургский г...,13.05.2019 09:58,Не указано
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4214620,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,системный администратор,"Уфа , не готов к переезду , не готов к команди...",полная занятость,"полный день, удаленная работа",Опыт работы 11 лет 5 месяцев Январь 2008 — по...,БГАУ,Заведующий отделом САПО,Высшее образование 2018 Башкирский государств...,09.04.2019 08:02,Имеется собственный автомобиль
4214621,49108929,Филиал ФКУ Налог-Сервис ФНС России в г. Москве,системный администратор,True,60000.0,NaN,От 1 года до 3 лет,Полный день,"['Windows 7', 'Windows 8', 'Windows 10', 'Unix...",Функциональные обязанности: Сопровождение опе...,...,системный администратор,"Санкт-Петербург , не готов к переезду , готов ...",полная занятость,"гибкий график, полный день, сменный график, уд...",Опыт работы 3 года 9 месяцев Системный админи...,"Российские железные дороги, ОАО",Инженер-электроник,Высшее образова

In [38]:
job_resume.columns

Index(['Ids', 'Employer', 'Name', 'Salary', 'From', 'To', 'Experience',
       'Schedule', 'Keys', 'Description', 'Area', 'Professional roles',
       'Specializations', 'Profarea names', 'Published at', 'index',
       'Пол, возраст', 'ЗП', 'Ищет работу на должность:',
       'Город, переезд, командировки', 'Занятость', 'График', 'Опыт работы',
       'Последнее/нынешнее место работы', 'Последняя/нынешняя должность',
       'Образование и ВУЗ', 'Обновление резюме', 'Авто'],
      dtype='object')

## Model

In [7]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [22]:
embeddings = model.encode(["Paris is the capital of France", "France is a nice country", "PC is good for gaming"])
embeddings.shape

(1, 384)

In [28]:
train_examples = []

for _, row in job_resume.iterrows():
    train_examples.append(InputExample(texts=[row["Description"], row["Опыт работы"]]))

In [29]:
train_examples[0].texts

['Компания «Комплексные Коммуникации» - более 15 лет комплексно разрабатывает и внедряет высокотехнологичные индивидуальные решения по построению систем промышленной связи и безопасности на градообразующих предприятиях и крупнейших транспортных узлах в РФ и странах СНГ. Центральный офис компании и производство расположены в Санкт-Петербурге Основная ценность компанииСотрудники, каждым из которых мы гордимся. Элита телекоммуникационной отрасли России и СНГ, которой приходится добиваться успеха в сложной, высокотехнологичной отрасли. В связи с развитием компании и увеличением числа клиентов открыта вакансия Менеджер проектных продаж Обязанности:  Ведение проектов по оснащению промышленных предприятий, транспортных узлов и сетей современными системами связи и безопасности. Работа с существующей клиентской базой, поддержка и развитие Участие в выставках, семинарах, конференциях. Работа с тендерами Командировки по РФ, странам СНГ, Западной Европе.  Требования:   Высшее профильное (желательн

In [30]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

num_epochs = 5
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [31]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
)

Epoch:   0%|          | 0/5 [04:18<?, ?it/s]


KeyboardInterrupt: 